## SearchEngine Class

In [4]:
# Data Manipulation
import numpy as np
import pandas as pd

class SearchEngine():
    def __init__(self, data):
        '''
        SearchEngine class
        Input: a songs dataset extracted from spotify API
        '''
        self.data = data

    def target_song(self, title='', artist=''):
        '''
        Search engine function for the target song
        Input: song title and/or artist
        Output: self.target dataset song
        '''
        # transform input strings in lowercase
        title = str(title).lower()
        artist = str(artist).lower()
        # filter self.data on the desired song
        if title != '' and artist != '':
            self.target = self.data[self.data[
                'artists'].str.lower().str.contains(artist)]
            self.target = self.target[self.target[
                'name'].str.lower().str.contains(title)]
        elif title != '':
            self.target = self.data[self.data[
                'name'].str.lower().str.contains(title)]
        elif artist != '':
            self.target = self.data[self.data[
                'artists'].str.lower().str.contains(artist)]
        else:
            print('Please select a song title and artist')
        # keep only the first song result
        self.target = self.target.head(1)
        self.artist = self.target["artists"].to_string(
            index=False).strip("['").strip("']")
        self.title = self.target["name"].to_string(index=False)
        print(f'TITLE: {self.title}')
        print(f'ARTIST: {self.artist}')

## Preprocessor Class

In [5]:
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.utils import shuffle

# Pipeline and Column Transformers
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn import set_config
set_config(display = "diagram")

# Scaling
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

# Package classes
# from music_similarity.search_engine import SearchEngine

class Preprocessor():
    def __init__(self, se):
        '''
        Preprocessor class
        Input: a songs dataset extracted from spotify API
        '''
        self.se = se

    def scale_data(self):
        '''
        Adapting data function
        '''
        # drop non numerical features before scaling
        self.X=self.se.data.drop(columns=['name','artists'])
        self.X_target=self.se.target.drop(columns=['name','artists'])
        # fit and transofrm with MinMaxScaler
        mmscaler = MinMaxScaler().fit(self.X)
        self.X_mmscaled=mmscaler.transform(self.X)
        self.X_target_mmscaled=mmscaler.transform(self.X_target)
        # fit and transofrm with RobustScaler
        roscaler = RobustScaler().fit(self.X)
        self.X_roscaled=roscaler.transform(self.X)
        self.X_target_roscaled=roscaler.transform(self.X_target)

## Playlist Class

In [6]:
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

# Unsupervised Learning
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# K-nn modelling
from sklearn.neighbors import NearestNeighbors

# Package classes
# from music_similarity.search_engine import SearchEngine
# from music_similarity.preprocessor import Preprocessor

class Playlist():
    def __init__(self, preprocessor, se):
        '''
        Extractor class
        Input: preprocessor class dataset extracted from spotify API
        '''
        self.preprocessor = preprocessor
        self.se = se
        self.playlist_songs = 10

    def build_model(self):
        '''
        Model builder function
        '''
        # Define the model
        self.model=NearestNeighbors(
            n_neighbors=self.playlist_songs + 1).fit(
            self.preprocessor.X_mmscaled)
        # Extract index and distance of self.playlist_songs+1
        # number of colest songs
        self.distance, self.index=self.model.kneighbors(
            self.preprocessor.X_target_mmscaled,
            n_neighbors=self.playlist_songs + 1)
        # Copy found index rows from the original not scaled dataset
        self.playlist = self.se.data.iloc[self.index[0],:]
        self.playlist['distance'] = self.distance[0]
        # Remove the target song from the list
        self.playlist = self.playlist.tail(self.playlist_songs)
        # Ordering the playlist on distance, ascending order
        self.playlist = self.playlist.sort_values(
            by=['distance'], ascending=True, ignore_index=True)
        # Drop not necessary columns
        self.playlist = self.playlist[['name', 'artists', 'distance']]
        # Strip square brackets from the artists strings
        self.playlist['artists'] = self.playlist['artists'].apply(
            lambda x: x.strip("['").strip("']"))
        # Set starting index from 0 to 1
        self.playlist.index += 1

In [7]:
if 'se' in globals():
    del se
spotify = pd.read_csv('../raw_data/ML_spotify_data.csv')
se = SearchEngine(spotify)
se.target_song("f", "u2")

if 'preprocessor' in globals():
    del preprocessor
preprocessor = Preprocessor(se)
preprocessor.scale_data()

if 'playlist' in globals():
    del playlist
playlist = Playlist(preprocessor, se)
playlist.build_model()

TITLE: A Sort Of Homecoming - Live
ARTIST: U2


/tmp/ipykernel_29067/2661942349.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.playlist['distance'] = self.distance[0]


In [8]:
playlist.playlist

,name,artists,distance
1,You've Got Another Thing Coming - Live from th...,Judas Priest,0.171471
2,Breaking The Silence - Remastered 2003,Queensrÿche,0.223612
3,"It Ain't Me, Babe - Live at LA Forum, Inglewoo...",Bob Dylan,0.232693
4,"Blasphemous Rumours - Live at Rose Bowl, Pasad...",Depeche Mode,0.236184
5,Feats Don't Fail Me Now - Live at Lisner Audit...,Little Feat,0.238395
6,Shoot Shoot - Live / 2008 Remaster,UFO,0.239608
7,"Nutrocker - Live At Newcastle City Hall, 26.3....","Emerson, Lake & Palmer",0.244874
8,Comin' Atcha Live / Truckin' - Live At The Tro...,Tesla,0.245215
9,Battle Angels,Sanctuary,0.269785
10,Baba O'Riley - Live At Shepperton,The Who,0.270837


## API DATA query

In [64]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

# TO DO: I'LL TURN THIS INTO A CLASS LATER

class ApiExtractor:
    def __init__(self, se):
        '''
        Extractor class
        Input: preprocessor class dataset extracted from spotify API
        '''
        # credentials need to be exported via the command line
        self.auth_manager = SpotifyClientCredentials()
        self.sp_connection = spotipy.Spotify(auth_manager=self.auth_manager)
        self.search_limit = 10
        self.se = se
    
    def get_track_attrs(self, artist, title):
        # get track attributes
        self.ta_response = self.sp_connection.search(
            q="artist:" + artist + " track:" + title,
            type="track",
            limit=1)

        # parse attributes from track
        self.track = self.ta_response['tracks']['items'][0]
        self.track_name = self.track['name']
        self.track_uri = self.track['uri']
        self.track_popularity = self.track['popularity']
        self.track_explicit = self.track['explicit']
        self.track_artists = [artist['name'] for artist in self.track['artists']]

        # get track features
        self.track_features = self.sp_connection.audio_features(tracks = self.track_uri)[0]

        return [self.track_name, self.track_artists, self.track_popularity,
                self.track_features['danceability'], self.track_features['valence'],
                self.track_features['energy'], self.track_explicit, self.track_features['key'],
                self.track_features['liveness'], self.track_features['loudness'],
                self.track_features['speechiness'], self.track_features['tempo']]
    
    def create_df_songs(self):
        '''
        Function that create self.df_songs dataframe
        with the following features:
        
        - uri 
        - name 
        - artists 
        - popylarity
        - explicit 
        features
        '''
        # extract the list of artists in a np array
        # artists_array from the baseline dataset
        self.artists_array = np.array([])
        for artist in self.se.data.artists.unique():
            self.artists_array = np.append(self.artists_array, artist)
        
        # Test array for debugging purposes
        # self.test_artists_array = np.array(["['Robin Trower', 'Jack Bruce', 'Bill Lordan']",   TEST
        #                                     "['Michael Hedges']"])                             TEST
         
        # find 10 most popular songs for each artist
        # creating features np arrays to store first 5 informations
        self.songs_uri_array = np.array([])
        self.songs_name_array = np.array([])
        self.songs_artists_array = np.array([])
        self.songs_popularity_array = np.array([])
        self.songs_explicit_array = np.array([])
        
        # Test array to not overload API requests
        # for artist in self.test_artists_array:                                                 TEST
        for artist in self.artists_array:
            # Perform API search request
            time.sleep(0.015)
            print(f'{artist} ...')
            # Try the query
            try:
                self.asl_response = self.sp_connection.search(
                                    q="artist:" + artist,
                                    type="track", 
                                    limit=self.search_limit)
            # If there is an exception:
            # jump to the next loop
            except:
                print(f"Oops! {artist} not valid.  Skipped...")
                continue
            
            # iteration over the self.search_limit songs
            for i in range(len(self.asl_response['tracks']['items'])):
                # append uri of the track
                self.songs_uri_array = np.append(
                    self.songs_uri_array, self.asl_response[
                        'tracks']['items'][i]['uri'])
                # append name of the track
                self.songs_name_array = np.append(
                    self.songs_name_array, self.asl_response[
                        'tracks']['items'][i]['name'])
                # append artists of the track
                self.songs_artists_array = np.append(
                    self.songs_artists_array, artist)
                # append popularity of the track
                self.songs_popularity_array = np.append(
                    self.songs_popularity_array, self.asl_response[
                        'tracks']['items'][i]['popularity'])
                # append explicit of the track
                self.songs_explicit_array = np.append(
                    self.songs_explicit_array, self.asl_response[
                        'tracks']['items'][i]['explicit'])

        self.df_songs = pd.DataFrame()
        self.df_songs['uri'] = self.songs_uri_array
        self.df_songs['name'] = self.songs_name_array
        self.df_songs['artists'] = self.songs_artists_array
        self.df_songs['popularity'] = self.songs_popularity_array
        self.df_songs['explicit'] = self.songs_explicit_array
        #self.df_songs.to_csv('../raw_data/songs.csv')
    
    def create_df_audio_analysis(self):
        '''
        Function that create a self.df_analysis dataframe 
        with the following features:
        
        - 12 pitch features
        - 12 timbre features 
        - uri 
        '''
        # From uri array list, query pitch and timbre features
        self.df_analysis = pd.DataFrame()
        self.df_songs_csv = pd.read_csv('../raw_data/songs.csv')
        self.songs_uri_array_csv = self.df_songs_csv['uri']
        # For testing purposes only iterate over the first 3 uri
        #for uri in self.songs_uri_array[:3]:
        #for uri in self.songs_uri_array:
        for uri in self.songs_uri_array_csv:
            time.sleep(0.015)
            print(f'{uri} ...')
            # Perform API audio_analysis request
            try:
                self.track_analysis = self.sp_connection.audio_analysis(track_id = uri)
            except:
                print(f"Oops! {uri} not valid.  Skipped...")
                continue
            # Reset segments arrays before new song analysis
            self.segment_pitch = []
            self.segment_timbre = []
            # Extract pitch and timbre for each segment of the song
            for segment in range(len(self.track_analysis['segments'])):
                self.segment_pitch.append(self.track_analysis['segments'][segment]['pitches'])
                self.segment_timbre.append(self.track_analysis['segments'][segment]['timbre'])
            # Reset df_song_segments dataframe
            self.df_song_segments = pd.DataFrame()
            # create pitch and timbre columns with np arrays
            self.df_song_segments['segment_pitch'] = self.segment_pitch
            self.df_song_segments['segment_timbre'] = self.segment_timbre
            # Split the array elements in different columns
            self.df_split_pitches = pd.DataFrame(self.df_song_segments['segment_pitch'].tolist(), columns=['sp1', 'sp2', 'sp3', 'sp4', 'sp5', 'sp6', 'sp7', 'sp8', 'sp9', 'sp10', 'sp11', 'sp12'])
            self.df_split_timbres = pd.DataFrame(self.df_song_segments['segment_timbre'].tolist(), columns=['tm1', 'tm2', 'tm3', 'tm4', 'tm5', 'tm6', 'tm7', 'tm8', 'tm9', 'tm10', 'tm11', 'tm12'])
            # Add new columns to df_segment
            self.df_song_segments = pd.concat([self.df_song_segments, self.df_split_pitches, self.df_split_timbres], axis=1)
            self.df_song_segments.drop(['segment_pitch', 'segment_timbre'], axis = 1, inplace = True)
            # Transpose mean serie into a dataframe row
            self.df_song_segments = self.df_song_segments.mean().to_frame().T
            # Add uri column to perform the future merge
            self.df_song_segments['uri'] = uri
            self.df_analysis = pd.concat([self.df_analysis, self.df_song_segments])
        #self.df_analysis.to_csv('../raw_data/analysis.csv')
    
    def create_df_audio_features(self):
        '''
        Function that create a self.df_features dataframe 
        with the following features:
        
        - danceability
        - energy
        - key
        - loudness
        - mode
        - speechiness
        - acousticness
        - instrumentalness
        - liveness
        - valence
        - tempo
        - uri
        '''
        self.df_features = pd.DataFrame()
        self.df_songs_csv = pd.read_csv('../raw_data/songs.csv')
        self.songs_uri_array_csv = self.df_songs_csv['uri']
        #for uri in self.songs_uri_array[:2]:
        for uri in self.songs_uri_array_csv:
            time.sleep(0.015)
            print(f'{uri} ...')
            # Perform API audio_features request
            try:
                self.track_features = self.sp_connection.audio_features(tracks = uri)[0]
            except:
                print(f"Oops! {uri} not valid.  Skipped...")
                continue
            self.df_track_features = pd.DataFrame()
            self.df_track_features = pd.DataFrame.from_dict([self.track_features])
            try:
                self.df_track_features.drop(columns=[
                'type',
                'track_href', 
                'analysis_url', 
                'duration_ms', 
                'time_signature', 
                'id'], inplace=True)
            except:
                print(f"Oops! {uri} has no features.  Skipped...")
            self.df_features = pd.concat([self.df_features, self.df_track_features])
        #self.df_features.to_csv('../raw_data/features.csv')
    
    def merge_dataframes(self):
        '''
        Function that merge all the dataframe toghether
        '''
        self.full_data = pd.merge(self.df_songs, self.df_analysis, how='outer', on='uri')
        self.full_data = pd.merge(self.full_data, self.df_features, how='outer', on='uri')
        

In [65]:
if 'ae' in globals():
    del ae
ae = ApiExtractor(se)
ae.create_df_audio_features()

spotify:track:0zK545STj6P7qbFSpCK9pp ...
spotify:track:6S3JlDAGk3uu3NtZbPnuhS ...
spotify:track:2XKW8CH8nRZH9cF2DNjBHN ...
spotify:track:54qG9hpUjLgkgTJQ9qvB1P ...
spotify:track:5CmGnAY7I7Ggj303HZBbkX ...
spotify:track:0JqDfzUZZi86BoZnx1MPQv ...
spotify:track:0wb21I1NV5uVUolRXIqlJF ...
spotify:track:3ccHstKEStIOANQsaw078p ...
spotify:track:0V8SWgdKA0BWydzwQflZAY ...
spotify:track:1VrIr1toXX6bOSdotodB71 ...
spotify:track:6zSpb8dQRaw0M1dK8PBwQz ...
spotify:track:3gdewACMIVMEWVbyb8O9sY ...
spotify:track:7rglLriMNBPAyuJOMGwi39 ...
spotify:track:1jDJFeK9x3OZboIAHsY9k2 ...
spotify:track:2TVxnKdb3tqe1nhQWwwZCO ...
spotify:track:5Wj1rJnCLpMHdLaxsFtJLs ...
spotify:track:7HW5WIw7ZgZORCzUxv5gW5 ...
spotify:track:38zsOOcu31XbbYj9BIPUF1 ...
spotify:track:6WCeFNVAXUtNczb7lqLiZU ...
spotify:track:4IRHwIZHzlHT1FQpRa5RdE ...
spotify:track:3LQPTJEqOfljGBxmpgUnoC ...
spotify:track:5lEyMg4GNKnMbaarvT4sRd ...
spotify:track:3beYmUcaXJZiSZMPbsMI7p ...
spotify:track:0emHuukZSuaOzOlsAWHj2W ...
spotify:track:29

spotify:track:4IXg5HtwR1KXWRYIzZBdeN ...
spotify:track:0DeT8WBqfwpvDuEkCyJH7V ...
spotify:track:1LdVhEybhMRi990GM8b5Qo ...
Oops! spotify:track:1LdVhEybhMRi990GM8b5Qo has no features.  Skipped...
spotify:track:2GyzcHu6OmpoV1rcIHBJQ5 ...
spotify:track:2GXuLNYlJGRb7SxZM2YlIb ...
spotify:track:7mjSHL2Eb0kAwiKbvNNyD9 ...
spotify:track:476V2d6iA2tWXgQboKmTtA ...
spotify:track:4d0d2llNcQIHmaCgYTrODQ ...
spotify:track:0wEo4mwfs6tXYgCem9m9wP ...
spotify:track:5y8d8RHvxb0KiTa9Nq0xWp ...
spotify:track:340t55txxJWQmTFK7wARZh ...
spotify:track:5p3nvmoaAY03qGmBQApn0w ...
spotify:track:7kTmndsuOxGut8Oy85Mn62 ...
spotify:track:7eiNxeOC8dvYAJWgTOu6xn ...
spotify:track:5zZqoeBLuXCDVdcjggDQdS ...
spotify:track:0MHXrqn909p0LRTPsNsGEi ...
spotify:track:7zrPswSV1yxrill5OyCuaU ...
spotify:track:4XsH9zBWPOCdXoH9ZDdS8r ...
spotify:track:01gDLZsi0j5fWC28FLPNO8 ...
spotify:track:2ouFIPC3O1HNBcVTw1d9AN ...
spotify:track:7gmB6hVMsv0XbjfiTgpATI ...
spotify:track:1Mq4vRRox3bcruJA75M4D9 ...
spotify:track:7J8F8mHNV79W

spotify:track:2HudfMAHpahXeWpgeZkk3r ...
spotify:track:4tSTpnw8UYNlroCEo774tm ...
spotify:track:7AU0zo4WShS29olDPT5DsN ...
spotify:track:6pqWrRF9K2PHpBmmRSIte4 ...
spotify:track:3toxxSU3TcvzgmeR8LLUiX ...
spotify:track:0W7SzJd9YNDiEUxxeKGdTU ...
spotify:track:5UuikgHTxSRFRnC0zXx10i ...
spotify:track:0MYNeFsKpzBHwrJGfWZysV ...
spotify:track:6MBZyLisOWTChcevgFqwVr ...
spotify:track:1JrNAxiq1o57wxuI9aX6KJ ...
spotify:track:1IQAsCKoXsEJ7tdI0dxTqT ...
spotify:track:2hSOrxX3I69A8xth8arEq2 ...
spotify:track:2it4DH8fqcCkSTCmqY5gzk ...
spotify:track:5bSSkaqVo0qsOmtYOcPpp3 ...
spotify:track:7bo2X63U1NpKYMXV1mZFkq ...
spotify:track:3rFAKoUoTUtREl0ob24L4h ...
spotify:track:5Zpy2HR2vYLjlTQVd81DLH ...
spotify:track:1oAr9H6Qw5gaqBJaZeMwS0 ...
spotify:track:5J7aILM0273RGHm7czdLxx ...
spotify:track:3v19aK0OgDXN1M1KAliHV3 ...
spotify:track:1TEvvaq5thKufLYdlwZV6d ...
spotify:track:4jU7QYKyxlsMCmioO6inQ3 ...
spotify:track:56GBItqD18Be9iN7zGqMX5 ...
spotify:track:6raOBJAoz2vcD2xByWJqqG ...
spotify:track:6H

spotify:track:4XDg07TABQlNRJcTDTuWLA ...
spotify:track:7lLIqYkXGKcpEeauufWlZm ...
spotify:track:60nl7K29XwEJtXCpEE57V4 ...
spotify:track:6xncjf74zeGIdP45l0rEUZ ...
spotify:track:4e41evlv5yGoGbxOa5Gmzh ...
spotify:track:5k4dL7PKD3iPaSsjsxp6lt ...
spotify:track:542C7PqR1oEfl63hlhQO4V ...
spotify:track:6d7Jz2np6jtNaZv1DiWfJs ...
spotify:track:6lMNJzDA47MaCZjjKgam5N ...
spotify:track:4NRFiS0pHjTFDjOm1LWTZj ...
spotify:track:5kXmiepRVuKhhz0SxyCVeL ...
spotify:track:0Tbr4K8UQbzvN5YzQTTwbI ...
spotify:track:0wmk0GFmNaox46CP2h5CGR ...
spotify:track:1FXpAf6Ed6vkBc6EkZHtGT ...
spotify:track:3RRKZ8a8SCtiPZ5qdK2Hos ...
spotify:track:7eOGMNm7MBXBg1m02WFYuj ...
spotify:track:5lc9L9FeLBwlJPgEbq9uEw ...
spotify:track:5WJU527RQNyMLuKecjsL8V ...
spotify:track:6SgQIoLn8kpu8J4wfwGWy8 ...
spotify:track:2w7R3CQgvl8PIKJlwtT9Mv ...
spotify:track:1S4igP7OpCUIEC5l75DssU ...
spotify:track:5aDf6IeBqqRBjoFqymYk1P ...
spotify:track:2X6qfjDzhKQA7UqhzyJrQP ...
spotify:track:2nqpN3cSmL0tTlJ724fE2a ...
spotify:track:3J

spotify:track:0mVEatHvxil3ZGJhL8zI4w ...
spotify:track:5fTwUw09NNgCIOkEKu3bGB ...
spotify:track:6LEiisZCK5EVhYB2kM3AFs ...
spotify:track:1phh5V7T5JAaiIww22ARZk ...
spotify:track:7HNFphOjDJpUGiseV5YvPZ ...
spotify:track:4w3tQBXhn5345eUXDGBWZG ...
spotify:track:2SpEHTbUuebeLkgs9QB7Ue ...
spotify:track:6dnco8haegnJYtylV26cBq ...
spotify:track:5fdhThPDe6jQQDqCyWrdAn ...
spotify:track:6H7nDglS6xWpRidbhMwI2L ...
spotify:track:6nRvgo4ySezQKD1D7NIqAu ...
spotify:track:1l2Xmg54EUr3TDzNIaCr1z ...
spotify:track:5oheUEVCpJllnImTs5nNiS ...
spotify:track:27876bQcDNkZhWqvF0zfFN ...
spotify:track:77bNe6jYY8yHdP7orXrz5I ...
spotify:track:2aoo2jlRnM3A0NyLQqMN2f ...
spotify:track:0wJoRiX5K5BxlqZTolB2LD ...
spotify:track:1Eolhana7nKHYpcYpdVcT5 ...
spotify:track:2AxCeJ6PSsBYiTckM0HLY7 ...
spotify:track:0NWPxcsf5vdjdiFUI8NgkP ...
spotify:track:3qebSLgUwaEqbwCM5sj5Kh ...
spotify:track:5H6Jp0syB5yEPk7SWYdlmk ...
spotify:track:23MrkN7g6Q5U7GLIxNHN1B ...
spotify:track:1ntxpzIUbSsizvuAy6lTYY ...
spotify:track:5L

...

spotify:track:5snCfoQbJ7S4cYB0K8XuJ4 ...
spotify:track:2n3qidzvO42pCcYS2yc1ZS ...
spotify:track:47RyOrXqLydmsPrsVU7nTw ...
spotify:track:7xDG0n0W0AV7oryFZes8rF ...
spotify:track:01bfGEk9QAfTky6HqqpWR9 ...
spotify:track:3UQSF4LRAqmoJmjXnuOle2 ...
spotify:track:21tTRofoXdyMJ5K31yX830 ...
spotify:track:58OLgaQqdVuRQMAjRTppGW ...
spotify:track:3YYdvweWofzy2QrrSiWAIf ...
spotify:track:3f3FDZajRpcLQGln7OZqdi ...
spotify:track:6aHFdm58mPF4YPJX3FgI0z ...
spotify:track:0QMmoBAY9pr7nOYisYWuMj ...
spotify:track:1bwnXBfIQFIBgBtsBADycD ...
spotify:track:65lzU29K7uXLHxJNI70d4Z ...
spotify:track:3Zj9WSEjueL8UecCOC67tT ...
spotify:track:732MYWbvyL6ss5rwl5g0dy ...
spotify:track:3dLiLNCzaheIT0GOCCPcnt ...
spotify:track:5UNTTIGYBLo0HBHljxVnFu ...
spotify:track:4zXqNHCmDvm2q7fGk2Detq ...
spotify:track:7yFhA2fUsL2oIMWlw5DaHQ ...
spotify:track:5HVc3YeNKDFomPhXBKE90U ...
spotify:track:1LM5zQv5pBKPyO7rm7Uz6U ...
spotify:track:1yTTMcUhL7rtz08Dsgb7Qb ...
spotify:track:09iyGiljLWRYXdRazFkdtJ ...
spotify:track:4H

spotify:track:2RIRXJ0ObiCH30ASGrHWfR ...
spotify:track:4OO9xZjfCOjP8kPUWh4VqW ...
spotify:track:3FAJ6O0NOHQV8Mc5Ri6ENp ...
spotify:track:6TSO4hwz9WGrEUY7ABGVim ...
spotify:track:6qBFSepqLCuh5tehehc1bd ...
spotify:track:61Emqg95O9zo1GNOcyxq4Y ...
spotify:track:7r1MqPAD09w24mCUUbxiCI ...
spotify:track:52wlhIULOeaxZpzuTYrVlt ...
spotify:track:1TuSfYnKU5uKbYK1esQDLj ...
spotify:track:1TlTzdsEpgfsmbuNfYmWfG ...
spotify:track:5PSYn0XVImstXPFFDtajEK ...
spotify:track:2OTRqsDes7OYRRwbF2JXsa ...
spotify:track:0DaMA9Qeod1l8d6kmEq4Ay ...
spotify:track:40GSMkmnXXD9rw1Rn73b5B ...
spotify:track:2z67brktck872cAvqDYcn6 ...
spotify:track:4IdlAwac0dkkQDoU5EfOSa ...
spotify:track:05RhYrd9EQMeLJUFelKmh6 ...
spotify:track:5pb5yLpX1nSscm1hvgXlUX ...
spotify:track:5DDLIuL5CLMeX9mOGoY5il ...
spotify:track:7wGoRwE6MZnm9flIMKWPul ...
spotify:track:0HQlgAyTXnZ9TJU7SA4xj6 ...
spotify:track:2WFhq6Mfe5Z2jYWTvkaKHi ...
spotify:track:4l4JMqBn3CS3xkP0oWndLa ...
spotify:track:3aQ1mUYRdJ2gKpbZ8T7tVS ...
spotify:track:0f

spotify:track:6H7RUDjmjiGV4I9hE24Kk5 ...
spotify:track:1DRBUVcJAIj6KiJRgOZV2I ...
spotify:track:0XBEeGWhlBEkAwkapdfSOK ...
spotify:track:6lE3B2UddJjKoZPPgdKvi8 ...
spotify:track:5IdUObvQ5gJQHgz9KgoBSg ...
spotify:track:4X0bwQ11ysUg0UqM2FuPaC ...
spotify:track:0FqLTCGVBCtrGI4ihCqqd9 ...
spotify:track:1j5TUQWSSQiPuKvHENwVsV ...
spotify:track:5jm8ylpH9gkjtyCGU2HCUW ...
spotify:track:3tHVSoQ3b8JnWofgVtcN84 ...
spotify:track:2To2iYqpHtesLvMdonPu8k ...
spotify:track:6s36KDFgCWW4lqDJ0njm7p ...
spotify:track:3ctaMit7CuiHIPVYrRvm15 ...
spotify:track:5DtVA6N0GjcVozR4NR8RzC ...
spotify:track:0XCtA9pYB0aOciPzrJpkAK ...
spotify:track:4zWHluAiPLx4tJ7T3K5vt8 ...
spotify:track:4x1ytzl1fpeWpmuRZHGnDu ...
spotify:track:0JwfzztXrwsgOoL8jNaKFT ...
spotify:track:7itKnwfqP5sLXmYp3vXWnR ...
spotify:track:76ZVDFSDmmIfmj8qpsHVJN ...
spotify:track:1Y6vzWgCoLDKCUnpeO0bRt ...
spotify:track:740kQ8xwSGloumdiKR5Cwz ...
spotify:track:5udcahJZJ9FFh1bFflKDZE ...
spotify:track:5rZlwNFl01HqLWBQGryKSm ...
spotify:track:5e

spotify:track:400mdliER9Oc9xA8TSDNvE ...
spotify:track:4SjQF4IpVlOCO4Qsvyaddl ...
spotify:track:7hg44Uac2HOBJCLWsHXMQp ...
spotify:track:7HuBQSNWUaYsPw5VYmoWtx ...
spotify:track:5GYJ82Vbwx3iwqUrz5oqhp ...
spotify:track:0veKH02hykLAW6zdlkeE4m ...
spotify:track:5eWMddYH4u3H7Ot4k9XQEO ...
spotify:track:2Hj8O5qBc2y2hQot2iCZQz ...
spotify:track:6CM1Y2Zk3LkqysmhnA5ij9 ...
spotify:track:2hX0U7zAyFbljltCfywyQC ...
spotify:track:7fXDP5wUtMQR6zd18DdfHc ...
spotify:track:35d6Wjch1dXIDK7dIyLDwu ...
spotify:track:3QHT5t8q72sx67GAEIAquN ...
spotify:track:0HIj0uo7zaFIpkzUIkDlmL ...
spotify:track:06s3QtMJVXw1AJX3UfvZG1 ...
spotify:track:5W83ErFkO3aKAIS1WMi6u0 ...
spotify:track:3f5lA8nnc4YYF1qf78WpHw ...
spotify:track:4xkQvF1yHZFN3lxDFj4qW1 ...
spotify:track:7k3qWn8D43a5ISa3x8lJux ...
spotify:track:2ksOAxtIxY8yElEWw8RhgK ...
spotify:track:0VJG6ELnHyaSyrUwZ3TK5a ...
spotify:track:3kSIkfNFz7lsswh2ZN7J2I ...
spotify:track:5khsSsyDLu8ibrCgaTpGDI ...
spotify:track:3bBCgAfJEAGMTQdtKFyPJJ ...
spotify:track:4D

spotify:track:4H3t8C59tXjKf1R2iKtc5M ...
spotify:track:2BaBwcllqDMkoD9184xOxP ...
spotify:track:7wnO6pBzcWWT8zT4vFx7hM ...
spotify:track:1HAvMItHMFRlnrbOXZBL8X ...
spotify:track:43UQAAZQB9eVlZhdz5pnt1 ...
spotify:track:6ajVb9JDdueaxbtLImUEPC ...
spotify:track:317vHXvCtvtAJIEbBJIuys ...
spotify:track:16WS0oVYnZMpl9ePTof7eH ...
spotify:track:6Yjh5hEKRWoQRM3ltyBOIt ...
spotify:track:6xGruZOHLs39ZbVccQTuPZ ...
spotify:track:0rKtyWc8bvkriBthvHKY8d ...
spotify:track:1VGzxJnVQND7Cg5H5wGj14 ...
spotify:track:0baNzeUcPQnQSagpe8T0mD ...
spotify:track:7wvwXi9Z66dcH5t202vVkN ...
spotify:track:6Ed1q0X8oSKSm4IIhiQbYg ...
spotify:track:5F1ZmucV0uJY14y2dYKePa ...
spotify:track:39oeKRgsLeynDIbWuXyA47 ...
spotify:track:0Y9D1Bc4fAkeZnVsGKdij6 ...
spotify:track:1MONUudxAjEk76FJvzGhuD ...
spotify:track:0XDA8IPloCFvf8c0ZNzRnB ...
spotify:track:0yU8ftsMI0LDMtee8TDsDR ...
spotify:track:4xpTM1MCKhY91OGiOhHNf5 ...
spotify:track:4Osv8sNkXqed1UUaPwoghd ...
spotify:track:5IZwWowYsN0wD6C1123ZmJ ...
spotify:track:2Y

In [47]:
ae.df_songs.to_csv('../raw_data/songs.csv')

In [55]:
ae.df_analysis.to_csv('../raw_data/analysis.csv')

In [66]:
ae.df_features.to_csv('../raw_data/features.csv')